In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import mlflow
import pytorch_lightning as pl
from pytorch_lightning.utilities.cloud_io import load as pl_load


In [3]:
from src.model import HemelingNet, HemelingRotNet
from src.classifier_net import BinaryClassifierNet, ExportValidation
from run_train import load_dataset, parse_config, setup_model

In [17]:
EXP = '/home/gaby/These/src/Fundus/RotInvariance/2-Scripts/experiments/baseline11/'
cfg = parse_config(EXP+'exp_config.yaml')
trainD, validD, testD = load_dataset(cfg)

In [14]:
model = setup_model(cfg.model)
net = BinaryClassifierNet(model=model)
state_dict = pl_load(EXP+cfg['experiment']['name']+'-{0:0=3d}/best-acc.ckpt'.format(25))['state_dict']
net.load_state_dict(state_dict)
net.eval()
None

In [20]:
tester = pl.Trainer(gpus=[0])
net.testset_names, data = list(zip(*testD.items()))
tester.test(net, data)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'MESSIDOR-acc/dataloader_idx_0': tensor(0.7607, device='cuda:0'),
 'MESSIDOR-iou/dataloader_idx_0': tensor(0.5366, device='cuda:0'),
 'MESSIDOR-roc/dataloader_idx_0': tensor(0.7918, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'HRF-acc/dataloader_idx_1': tensor(0.6614, device='cuda:0'),
 'HRF-iou/dataloader_idx_1': tensor(0.4214, device='cuda:0'),
 'HRF-roc/dataloader_idx_1': tensor(0.7580, device='cuda:0')}
--------------------------------------------------------------------------------
DATALOADER:2 TEST RESULTS
{'DRIVE-acc/dataloader_idx_2': tensor(0.7131, device='cuda:0'),
 'DRIVE-iou/dataloader_idx_2': tensor(0.4840, device='cuda:0'),
 'DRIVE-roc/dataloader_idx_2': tensor(0.8182, device='cuda:0')}
--------------------------------------------------------------------------------


[{'MESSIDOR-acc/dataloader_idx_0': 0.7607135772705078,
  'MESSIDOR-roc/dataloader_idx_0': 0.7918192744255066,
  'MESSIDOR-iou/dataloader_idx_0': 0.5366097092628479},
 {'HRF-acc/dataloader_idx_1': 0.6613824367523193,
  'HRF-roc/dataloader_idx_1': 0.75798100233078,
  'HRF-iou/dataloader_idx_1': 0.4213772118091583},
 {'DRIVE-acc/dataloader_idx_2': 0.7130728363990784,
  'DRIVE-roc/dataloader_idx_2': 0.8181764483451843,
  'DRIVE-iou/dataloader_idx_2': 0.4840429425239563}]

collections.OrderedDict

In [ ]:
for batch in testD['DRIVE']:
    r = model(batch['x'])
    break

In [ ]:
plt.imshow(r[0,0].detach())

In [ ]:
r['ypred'].shape

In [ ]:
print(cfg.model['rot-eq'])